# Clean Data

In [1]:
import pandas as pd
import numpy as np
import datetime
import copy
import os
import re

today = datetime.date.today()

In [4]:
# chunks = pd.read_csv('../raw_data/scraped_data.csv', chunksize=10000)
chunks = pd.read_csv('../backup_data/backup_data_14.csv', chunksize=10000)  # data copied from desktop

df = pd.concat(chunks, ignore_index=True)

df.to_csv('../backup_data/backup_data_{}.csv'.format(len(os.listdir('../backup_data/'))))  # create back up of csv

df.shape

(4409663, 16)

In [5]:
try:
    df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    pass

In [6]:
df.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title
0,jonashendrickx,NaN,347,/r/thinkpad/comments/74drfl/thinkpad_25_discus...,0,NaN,/r/thinkpad/comments/74drfl/thinkpad_25_discus...,t3_74drfl,1507178424000,87.0,1.509491e+09,r/thinkpad,3637.0,21512.0,ThinkPad 25 Discussion Thread (Release Oct 5) ...
1,MercGaberkles,NaN,4,https://i.redd.it/bwujg5tvn7vz.jpg,0,NaN,/r/thinkpad/comments/79xuvy/happy_halloween_rt...,t3_79xuvy,1509475415000,50.0,1.509491e+09,r/thinkpad,3637.0,21512.0,Happy Halloween r/ThinkPad! (i.redd.it)
2,dan988,NaN,68,https://i.redd.it/vqqssqp9k5vz.jpg,0,NaN,/r/thinkpad/comments/79v62u/swapped_out_an_xps...,t3_79v62u,1509450227000,63.0,1.509491e+09,r/thinkpad,3637.0,21512.0,Swapped out an XPS13 for this! (i.redd.it)
3,MUnderwoodBarcode,NaN,5,https://imgur.com/gallery/pxGGU,0,NaN,/r/thinkpad/comments/79yy81/spotted_joe_rogans...,t3_79yy81,1509484723000,NaN,1.509491e+09,r/thinkpad,3637.0,21512.0,Spotted Joe Rogan's TP (imgur.com)
4,Hotshot9146,NaN,3,https://imgur.com/gallery/cBj6E,0,NaN,/r/thinkpad/comments/79z1d7/mint_x61_for_now_ʖ...,t3_79z1d7,1509485487000,NaN,1.509491e+09,r/thinkpad,3637.0,21512.0,"Mint X61... for now ( ͡~ ͜ʖ ͡°) $60 for it, de..."


In [7]:
print('Subreddits: {}'.format(len(df['subreddit'].unique())))
print(df['subreddit'].unique())

Subreddits: 2923
['r/thinkpad' 'r/IWantOut' 'r/simpleliving' ..., 'r/Eve' 'r/ravens'
 'r/Emma_Roberts']


In [8]:
print('Users: {}'.format(len(df['author'].unique())))
print(df['author'].unique())

Users: 1049005
['jonashendrickx' 'MercGaberkles' 'dan988' ..., 'GeoTrip' 'ratchetassjimmy'
 'ryeryebaby']


In [9]:
df = df.drop_duplicates(['author', 'post_id'], keep='last')
df = df[df['author'] != 'AutoModerator']  # remove posts from the moderator bot
df.shape

(2590769, 15)

In [12]:
df.comments.sum()

42374242

In [10]:
=

SyntaxError: invalid syntax (<ipython-input-10-9ae22869553b>, line 1)

In [ ]:
df.describe()

In [ ]:
df['author'].value_counts().head(10)

In [ ]:
df['title'] = [re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", title)).replace('()', '') for title in df['title'].values]

In [ ]:
from emoji import UNICODE_EMOJI

type(UNICODE_EMOJI)

def has_emoji(text):
    emoji = 0
    try:
        for i in text:
            if i in UNICODE_EMOJI.keys():
                emoji = 1
                break
    except:
        pass
    return emoji
        
    
df['has_emoji'] = df['title'].apply(has_emoji)

In [ ]:
df[df['has_emoji'] == 1].head(1)

In [ ]:
def outside_content(content):
    outside = 0
    if content[1] != 'r':
        outside = 1
    return outside

df['outside_content'] = df['content'].apply(outside_content)

In [ ]:
df.head()

In [ ]:
def is_question(title):
    question = 0
    if '?' in title:
        question = 1
    return question

df['is_question'] = df['title'].apply(is_question)

In [ ]:
df.head()

In [ ]:
df['scrape_time'] = df['scrape_time'].apply(datetime.datetime.fromtimestamp)

In [ ]:
df['post_time'] = df['post_time']/1000
df['post_time'] = df['post_time'].apply(datetime.datetime.fromtimestamp)

df['elapsed_time'] = df['scrape_time'] - df['post_time']
df['minutes_since_post'] = df['elapsed_time'].dt.total_seconds()/60

In [ ]:
defaults = ['r/announcements/', 'r/Art', 'r/AskReddit', 'r/askscience', 'r/aww', 'r/blog', 'r/books', 'r/creepy',
            'r/dataisbeautiful', 'r/DIY', 'r/Documentaries', 'r/EarthPorn', 'r/explainlikeimfive', 'r/food', 
            'r/funny', 'r/Futurology', 'r/gadgets', 'r/gaming', 'r/GetMotivated', 'r/gifs', 'r/history', 'r/IAmA',
            'r/InternetIsBeautiful', 'r/Jokes', 'r/LifeProTips', 'r/listentothis', 'r/mildlyinteresting', 'r/movies',
            'r/Music', 'r/news', 'r/nosleep', 'r/nottheonion', 'r/OldSchoolCool', 'r/personalfinance', 'r/philosophy',
            'r/photoshopbattles', 'r/pics', 'r/science', 'r/Showerthoughts', 'r/space', 'r/sports', 'r/television',
            'r/tifu', 'r/todayilearned', 'r/UpliftingNews', 'r/videos', 'r/worldnews']

def default_subreddit(subreddit):
    is_default = 0
    if subreddit in defaults:
        is_default = 1
    return is_default

df['default_subreddit'] = df['subreddit'].apply(default_subreddit)

In [ ]:
df[df['default_subreddit'] == 1].head(1)

In [ ]:
if not os.path.isfile('../cleaned_data/cleaned_data.csv'):
    print("\nNEW FILE\n")
    df.to_csv('../cleaned_data/cleaned_data.csv', index=False)
else:
    df.to_csv('../cleaned_data/cleaned_data.csv', mode='a', index=False, header=False)

In [ ]:
clean_data = pd.read_csv('../cleaned_data/cleaned_data.csv')

clean_data.sort_values('scrape_time', inplace=True)

clean_data.drop_duplicates('post_id', keep='last', inplace=True)

In [ ]:
clean_data.shape

In [ ]:
clean_data.to_csv('/data/reddit_data.csv', index=False)